In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image

In [ ]:
# Dataset directory path
dataset_dir = 'C:/Users/manish kumar/Desktop/ML Project/Testing'

# Image dimensions (resize images to this size)
img_width, img_height = 224, 224

# Emotion map (label encoding for emotions)
emotion_map = {'Angry': 0, 'Fear': 1, 'Happy': 2, 'Neutral': 3, 'Sad': 4, 'Suprise': 5}

# Initialize lists to store images and labels
images = []
labels = []

# Loop through each emotion folder
for emotion, label in emotion_map.items():
    emotion_folder = os.path.join(dataset_dir, emotion)
    for img_name in os.listdir(emotion_folder):
        img_path = os.path.join(emotion_folder, img_name)
        
        # Load and resize image
        img = image.load_img(img_path, target_size=(img_width, img_height))
        img_array = image.img_to_array(img)
        images.append(img_array)
        labels.append(label)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize the pixel values to be between 0 and 1
images = images / 255.0

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [ ]:
# Define ImageDataGenerator for training data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define ImageDataGenerator for validation and test data (only rescaling)
test_datagen = ImageDataGenerator(rescale=1./255)

# Fit the training data augmentation on the training data
train_datagen.fit(X_train)


In [ ]:
# Define the CNN model
model = Sequential()

# Convolutional layers with max pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten and add fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))

# Dropout to reduce overfitting
model.add(Dropout(0.5))

model.add(Dense(len(emotion_map), activation='softmax'))  # Output layer with softmax for multi-class classification

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


c:\Users\manish kumar\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# EarlyStopping callback to stop training when validation accuracy doesn't improve
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model using augmented data
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32),  # Training data with augmentation
    validation_data=test_datagen.flow(X_test, y_test),   # Test data (only rescaled)
    epochs=10,
    callbacks=[early_stop],  # Stop training early if needed
    verbose=1
)


c:\Users\manish kumar\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 97s 540ms/step - accuracy: 0.2448 - loss: 1.7722 - val_accuracy: 0.2405 - val_loss: 1.7697
Epoch 2/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 107s 607ms/step - accuracy: 0.2629 - loss: 1.7620 - val_accuracy: 0.2405 - val_loss: 1.7675
Epoch 3/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 98s 551ms/step - accuracy: 0.2592 - loss: 1.7576 - val_accuracy: 0.2405 - val_loss: 1.7681
Epoch 4/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 96s 544ms/step - accuracy: 0.2511 - loss: 1.7640 - val_accuracy: 0.2405 - val_loss: 1.7674
Epoch 5/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 97s 549ms/step - accuracy: 0.2528 - loss: 1.7659 - val_accuracy: 0.2405 - val_loss: 1.7674
Epoch 6/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 101s 573ms/step - accuracy: 0.2481 - loss: 1.7660 - val_accuracy: 0.2405 - val_loss: 1.7678
Epoch 7/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 186s 1s/step - accuracy: 0.2575 - loss: 1.7586 - val_accuracy: 0.2405 - val_loss: 1.7679
Epoch 8/10
177/177 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.2492 - loss: 1.7